In [1]:
# You've built a model. But how good is it?
# Measuring model quality is the key to iteratively improving your models.
# Validate = Doğrulamak

In [2]:
# Model değerlendirme sürecinde, çoğu zaman model kalitesinin ölçüsü olarak tahmin doğruluğu kullanılır. 
# Bu, modelin yaptığı tahminlerin gerçek sonuçlara ne kadar yakın olduğunu ifade eder. 
# Ancak, birçok kişi tahmin doğruluğunu ölçerken büyük bir hata yapar: 
# Eğitim verileriyle tahminler yapar ve bu tahminleri eğitim verilerindeki hedef değerlerle karşılaştırır. 
# Bu yaklaşım sorunludur, çünkü modelin genel performansını doğru bir şekilde yansıtmaz. 
# Bu hatanın neden problemli olduğunu ve nasıl çözülebileceğini anlayacaksınız.

In [3]:
# Model kalitesini anlaşılır bir şekilde özetlemeniz gerekir. 
# Örneğin, 10.000 ev için tahmin edilen ve gerçek ev değerlerini karşılaştırdığınızda, 
# muhtemelen iyi ve kötü tahminlerin bir karışımını bulacaksınız.
# 10.000 tahmin ve gerçek değerin listesini incelemek anlamsız olurdu. Bu nedenle, bunları tek bir metrikte özetlemek gerekir.

In [4]:
# Model kalitesini özetlemek için birçok metrik vardır, ancak Ortalama Mutlak Hata (Mean Absolute Error - MAE) ile başlayacağız. 
# Bu metriği anlamak için önce hata kavramını ele alalım.
# Her bir ev için tahmin hatası şu şekilde hesaplanır:

# hata = gerçek değer − tahmin edilen değer 
# Örneğin, bir evin değeri 150.000 dolar ise ve siz 100.000 dolar olarak tahmin ettiyseniz, hata 50.000 dolardır.

# MAE metriğinde, her hatanın mutlak değeri alınır, bu da tüm hataları pozitif sayılara çevirir. Daha sonra bu mutlak hataların ortalaması alınır. 
# Bu, model kalitesini ölçer ve basitçe şu şekilde ifade edilebilir:
# "Ortalama olarak, tahminlerimiz yaklaşık X kadar yanılıyor."
# MAE'yi hesaplamak için önce bir modele ihtiyacımız var. Bu model, aşağıdaki gizli hücrede oluşturulmuştur ve kodu inceleyerek görebilirsiniz.

In [6]:
import pandas as pd

# Load data
melbourneData = "C:/Users/Alperen Arda/OneDrive/Desktop/GitHub/Data-Science-and-Machine-Learning-Tutorial/Datasets/melb_data.csv"
data_houses = pd.read_csv(melbourneData)
# Filter rows with missing price values
filtered_melbourne_data = data_houses.dropna(axis=0)
# Choose target and features
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.tree import DecisionTreeRegressor
# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(X, y)

DecisionTreeRegressor()

In [5]:
from sklearn.metrics import mean_absolute_error

In [8]:
predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

434.71594577146544

In [9]:
# "In-sample" skorlar, modeli oluşturmak ve değerlendirmek için aynı veri setini kullanarak hesaplanan ölçümlerdir. 
# Bu yaklaşım yanıltıcı olabilir çünkü model, sadece eğitim verilerinde görülen örüntüleri öğrenir ve bu örüntüler gerçek dünyada geçerli olmayabilir. 
# Örneğin, eğitim verilerinde yeşil kapılı evler pahalı olabilir, bu da modelin her yeşil kapılı ev için yüksek fiyat tahmin etmesine neden olur. 
# Ancak, bu örüntü gerçek dünyada geçerli değilse, model yeni verilerle karşılaştığında hatalı tahminler yapar.

In [10]:
# Modellerin pratik değeri, yeni veriler üzerinde doğru tahminler yapabilmesinden gelir. 
# Bu nedenle, performansı, modelin oluşturulmasında kullanılmayan verilerle ölçeriz.******************************
# Bunu yapmanın en basit yolu, bazı verileri model oluşturma sürecinden hariç tutmak ve bu verileri modelin daha önce görmediği verilerle test etmek için kullanmaktır. 
# Bu verilere doğrulama verisi(validation data) denir.

In [ ]:
# The scikit-learn library has a function train_test_split to break up the data into two pieces. 
# We'll use some of that data as training data to fit the model, and we'll use the other data as validation data to calculate mean_absolute_error.

In [12]:
from sklearn.model_selection import train_test_split

In [15]:
# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)
# Eğer 0 yerine 243234 yazarsanız, veriler farklı bir şekilde bölünür, çünkü random_state değeri değiştiğinde, veri bölünmesi de rastgele olarak farklılaşır. 
# Ancak aynı random_state değeriyle tekrar çalıştırırsanız, her seferinde aynı şekilde bölünür. Bu, sonuçlarınızın yeniden üretilebilir olmasını sağlar. 
# Farklı random_state değerleri farklı veri bölünmeleri oluşturur, bu da modelin performansını biraz etkileyebilir, ancak genel süreç aynı kalır.
melbourne_model = DecisionTreeRegressor()
melbourne_model.fit(train_X, train_y)

DecisionTreeRegressor()

In [16]:
val_predictions = melbourne_model.predict(val_X)

In [17]:
print(mean_absolute_error(val_y, val_predictions))

262665.6081342802


In [ ]:
# İç veri setindeki ortalama mutlak hatanız yaklaşık 500 dolardı, ancak dış veri setinde bu hata 250.000 dolardan fazla çıktı. 
# Bu, neredeyse tamamen doğru bir model ile pratikte kullanılamayacak bir model arasındaki farkı gösterir. 
# Karşılaştırma açısından, doğrulama verilerindeki ortalama ev değeri 1.1 milyon dolar, yani yeni verilerdeki hata, ortalama ev değerinin yaklaşık dörtte biri kadardır.
# Bu modeli iyileştirmenin birçok yolu vardır, örneğin, daha iyi özellikler bulmak veya farklı model türlerini denemek gibi.

# kısacası validation verileri kullanmassak kendimizi kandırırız. validation gibi yöntemler bize modelin gerçek doğruluğunu verir. 
# Bu model 262k dolar hata ile kendini bok gibi olma konusunda kanıtlamıştır.